In [53]:
import pandas as pd
import os
import numpy as np
from wordcloud import WordCloud
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [56]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/train_data/train.csv', header = None)
# df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/test_data/test.csv', header=None)
# df_test = df.rename({0: 'label', 1: 'text'}, axis = 1)
df = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [58]:
df = df['text']

In [59]:
df = pd.DataFrame(df)

In [60]:
df.head()

,text
0,"Unfortunately, the frustration of being Dr. Go..."
1,Been going to Dr. Goldberg for over 10 years. ...
2,I don't know what Dr. Goldberg was like before...
3,I'm writing this review to give you a heads up...
4,All the food is great here. But the best thing...


In [61]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

In [62]:
df_text = df.text

In [63]:
df_text = pd.DataFrame(df_text)

In [64]:
df_text

,text
0,"Unfortunately, the frustration of being Dr. Go..."
1,Been going to Dr. Goldberg for over 10 years. ...
2,I don't know what Dr. Goldberg was like before...
3,I'm writing this review to give you a heads up...
4,All the food is great here. But the best thing...
...,...
559995,Ryan was as good as everyone on yelp has claim...
559996,Professional \nFriendly\nOn time AND affordabl...
559997,Phone calls always go to voicemail and message...
559998,Looks like all of the good reviews have gone t...


In [65]:
len_train = int(10/100*len(df_text['text']))
df_text = df_text['text']
df_text = df_text[:len_train]

In [66]:
df_text = pd.DataFrame(df_text)

In [67]:
df_text['clean_text'] = df_text.text.apply(clean)
df_text['clean_text'] = df_text['clean_text'].astype('str')

In [68]:
df_text

,text,clean_text
0,"Unfortunately, the frustration of being Dr. Go...","['unfortunately', 'frustration', 'dr', 'goldbe..."
1,Been going to Dr. Goldberg for over 10 years. ...,"['going', 'dr', 'goldberg', 'year', 'think', '..."
2,I don't know what Dr. Goldberg was like before...,"['know', 'dr', 'goldberg', 'like', 'moving', '..."
3,I'm writing this review to give you a heads up...,"['writing', 'review', 'give', 'head', 'see', '..."
4,All the food is great here. But the best thing...,"['food', 'great', 'best', 'thing', 'wing', 'wi..."
...,...,...
55995,I took a party of 6 friends and co-workers the...,"['took', 'party', 'friend', 'co', 'worker', 'l..."
55996,Stop by for lunch based on Yelp reviews. Tasty...,"['stop', 'lunch', 'based', 'yelp', 'review', '..."
55997,Best persian food ever.\nWOW!\n\nPossibly the ...,"['best', 'persian', 'food', 'ever', 'nwow', 'n..."
55998,"Habibbbbbb is the rudest, meanest, uglyist, gu...","['habibbbbbb', 'rudest', 'meanest', 'uglyist',..."


In [69]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(df_text['clean_text'])

lda_model = LatentDirichletAllocation(n_components=3)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [36]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

In [37]:
print_topics(lda_model, vectorizer)

Topic 0:
[('place', 8.304408244297012), ('food', 7.2839309646861965), ('like', 5.229908612708771), ('restaurant', 2.2501345473192442), ('ripped', 2.249826955583492), ('worst', 2.249789990173925), ('found', 2.249759044912799), ('delicious', 2.249694723126523), ('amazing', 2.246201682670819), ('never', 2.245548352429745)]
Topic 1:
[('like', 3.2552473101840214), ('never', 3.2542074205515674), ('service', 3.238949213629143), ('customer', 2.263019747740604), ('ever', 2.2499159234942154), ('still', 2.249900485708298), ('care', 2.2498887724529624), ('also', 2.2498318167045452), ('place', 2.195283831088168), ('back', 1.266797087531936)]
Topic 2:
[('good', 3.2500571968787937), ('really', 3.249877039650768), ('salad', 2.261752052476135), ('party', 2.2532704806375747), ('tender', 2.251341805302101), ('greek', 2.2499248396107783), ('bye', 2.249859750025168), ('damn', 2.2498280706053486), ('like', 2.2488985653514), ('time', 2.2484185876633953)]
Topic 3:
[('good', 8.149608302896468), ('great', 7.272